In [ ]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.model_selection import train_test_split
from ode_generators import generate_damped_pendulum_solution

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras import activations

%config InlineBackend.figure_format = 'retina'
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.style.use('seaborn-whitegrid')

In [ ]:
t, x, dxdt = generate_damped_pendulum_solution(one_step=False, α=0.1, β=8.91, t_end=25)

In [ ]:
plt.figure(figsize=(16,7))

plt.subplot(1,2,1)
plt.title("State Trajectory")
plt.plot(t, x, "-", label="x")
plt.plot(t, dxdt, "-", label="dxdt")
plt.xlabel("t")
plt.legend()

plt.subplot(1,2,2)
plt.title("Phase Space")
plt.plot(x, dxdt, "-")
plt.show()

In [ ]:
N_SAMPLES = 25000
TEST_SIZE = 0.2
X, Y = generate_damped_pendulum_solution(N_SAMPLES=N_SAMPLES, α=0.1, β=8.91, Δ=0.1, one_step=True)
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=TEST_SIZE, random_state=42)

In [ ]:
TRAIN_LENGTH = int(N_SAMPLES * (1-TEST_SIZE))
BATCH_SIZE = 10
BUFFER_SIZE = TRAIN_LENGTH
STEPS_PER_EPOCH = int(TRAIN_LENGTH // BATCH_SIZE)
VALIDATION_STEPS = int((N_SAMPLES*TEST_SIZE)//BATCH_SIZE)

In [ ]:
train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid.batch(BATCH_SIZE)

In [ ]:
def resnet_block(input_dim):
    inputs = keras.layers.Input(shape=(input_dim,))
    
    x = Dense(30, activation="tanh", kernel_initializer='random_normal', bias_initializer='zeros')(inputs)
    x = Dense(30, activation="tanh", kernel_initializer='random_normal', bias_initializer='zeros')(x)
    x = Dense(2, kernel_initializer='random_normal', bias_initializer='zeros')(x)
    
    x = layers.Add()([x, inputs])
    outputs = Dense(2, kernel_initializer='random_normal', bias_initializer='zeros')(x)
    
    model =  keras.Model(inputs=inputs, outputs=outputs)   
    return model
    

model = resnet_block(input_dim=2)

In [ ]:
# model.summary()
plot_model(model, show_shapes=True, dpi=64)

In [ ]:
# reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=1e-4, verbose=2)

In [ ]:
model.compile(
    loss=keras.losses.MeanSquaredError(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.001), 
)

In [ ]:
%%time

history = model.fit(
    train_dataset, 
    epochs=500, 
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
    validation_data = valid_dataset,
    verbose=2, 
    #callbacks=[reduce_lr]
)

In [ ]:
def plot_history(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    #learning_rate = history.history["lr"]
    epochs = range(len(loss))
    plt.figure(figsize=(15,7))
    plt.plot(epochs, loss, 'b', label="Training")
    plt.plot(epochs, val_loss, 'g', label="Validation")
    #plt.plot(epochs, learning_rate, 'r', label="Learning Rate")
    plt.title('Training and Validation Loss',fontsize=22)
    plt.yscale("log")
    plt.xlabel("Epoch", fontsize=20)
    plt.legend(prop={"size":20})
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
def predict_solution(Δ=0.1, t_end=20):

    t_steps = np.arange(0,t_end, Δ)
    X_pred = np.zeros((len(t_steps), 2))

    x_0 = np.array([-1.193, -3.876])
    x_Δ = np.array([-1.193, -3.876])
    
    for i in range(0, len(t_steps)):
        x_0 = x_Δ
        x_Δ = model.predict(np.expand_dims(x_0, axis=0))
        x_Δ = np.squeeze(x_Δ)
        X_pred[i] = x_Δ   
        
    t_steps = t_steps + Δ
    return t_steps, X_pred

In [ ]:
def get_error(t_actual, x_actual, t_pred, x_pred, t_end):
    Nt = t_actual.shape[0]
    errors = []
    for i in range(0,len(t_pred)):
        t_idx = int((Nt/t_end)*t_pred[i] - 1)
        x_true = x_actual[t_idx]
        x_est = x_pred[i]
        err = np.abs((x_true - x_est)/x_true)*100
        #print("x true = {:.4f}  x est = {:.4f}  error = {:.4f}".format(x_true, x_est, err))
        errors.append(err)
    return errors

In [ ]:
Δ=0.1
t_end=20

In [ ]:
t_steps, X_pred = predict_solution(Δ=Δ, t_end=t_end)   
t, x, dxdt = generate_damped_pendulum_solution(one_step=False, α=0.1, β=8.91, Δ=Δ, t_end=t_end)

In [ ]:
plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.title("State Trajectory", fontsize=20)
plt.plot(t, x, "-", c="r", label="Reference x")
plt.plot(t, dxdt, "-", c="r", label="Reference dxdt")
plt.scatter(t_steps, X_pred[:,0], s=40, c="b", marker="*", label="Approximation x")
plt.scatter(t_steps, X_pred[:,1], s=40, c="b", marker=".", label="Approximation dxdt")
plt.legend()

plt.subplot(1,2,2)
plt.title("State Space", fontsize=20)
plt.plot(x, dxdt, "-", c="r", label="Reference")
plt.scatter(X_pred[:,0], X_pred[:,1], s=40, c="b", marker=".", label="Approximation")
plt.legend()
plt.tight_layout()
plt.show()